In [30]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes
import re

In [31]:
stop_words = ['a', 'able', 'about', 'across', 'after', 'all', 'almost', 'also',
              'am', 'among', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because',
              'been', 'but', 'by', 'can', 'cannot', 'could', 'dear',
              'did', 'do', 'does', 'either', 'else', 'ever', 'every', 'for',
              'from', 'get', 'got', 'had', 'has', 'have', 'he', 'her', 'hers',
              'him', 'his', 'how', 'however', 'i', 'if', 'in', 'into', 'is',
              'it', 'its', 'just', 'least', 'let', 'like', 'likely', 'may',
              'me', 'might', 'most', 'must', 'my', 'neither', 'no', 'nor',
              'not', 'of', 'off', 'often', 'on', 'only', 'or', 'other', 'our',
              'own', 'rather', 'said', 'say', 'says', 'she', 'should', 'since',
              'so', 'some', 'than', 'that', 'the', 'their', 'them', 'then',
              'there', 'these', 'they', 'this', 'tis', 'to', 'too', 'twas', 'us',
              've', 'wants', 'was', 'we', 'were', 'what', 'when', 'where', 'which',
              'while', 'who', 'whom', 'why', 'will', 'with', 'would', 'yet',
              'you', 'your']

In [32]:
def parse(line):
    line = line.split('\n')
    line = [re.sub('[^A-Za-z]', ' ', i) for i in line]
    line = [i.split() for i in line]
    line = [item for sublist in line for item in sublist]
    line = [word.strip().lower() for word in line if word not in stop_words if len(word) >= 3]
    return line

In [33]:
pData = sc.textFile("C:\Users\KARAN\Documents\BDA\pos_data.txt")
ptData = pData.map(lambda text : LabeledPoint(1, HashingTF().transform(parse(text))))
print "No. of Positive Sentences: " + str(ptData.count())
ptData.persist()

No. of Positive Sentences: 5331


PythonRDD[57] at RDD at PythonRDD.scala:43

In [40]:
nData = sc.textFile("C:\Users\KARAN\Documents\BDA\data_neg.txt")
ntData = nData.map(lambda text : LabeledPoint(0, HashingTF().transform(parse(text))))
print "No. of Negative Sentences: " + str(ntData.count())
ntData.persist()

No. of Negative Sentences: 5331


PythonRDD[78] at RDD at PythonRDD.scala:43

In [41]:
ptrain, ptest = ptData.randomSplit([0.7, 0.3])
ntrain, ntest = ntData.randomSplit([0.7, 0.3])
trainh = ptrain.union(ntrain)
testh = ptest.union(ntest)

In [42]:
model = NaiveBayes.train(trainh)

In [44]:
prediction_and_labels = testh.map(lambda point: (model.predict(point.features), point.label))
correct = prediction_and_labels.filter(lambda (predicted, actual): predicted == actual)
accuracy = correct.count() / float(testh.count())
print "Classifier correctly predicted category " + str(accuracy * 100) + " percent of the time"

Classifier correctly predicted category 77.1735791091 percent of the time
